In [1]:
import numpy as np
import pandas as pd
from pandas import merge 
from os import listdir
from numpy import genfromtxt

## Get data

In [2]:
studyname = 'Utrecht' 

In [3]:
# function for Utrecht data to create list with filenames that need to be
# processed filtered by epoch and diagnosis
def select_filenames(datadir,labloc): 
    # Input:
    # - datadir: the path where the files are located
    # - labloc: the path and filename of the labels with columns: id, gender, age, diagnosis
    # Output:
    # - List of filenames for which filename ends with ep1.TRC.txt and diganosis < 3
    #======================================================================
    filenames = listdir(datadir) 
    fn = pd.DataFrame(filenames,columns=['filename'])
    fn2 = fn.filename.str.split('_',expand=True)
    fn2.columns= ['id','epoch']
    fn2['filename'] = fn
    fn2.id = pd.to_numeric(fn2.id)
    fn2 = fn2[(fn2['epoch'] == 'ep1.TRC.txt')]  
    y = pd.read_csv(labloc, sep=',',header=0)
    yp = pd.DataFrame(y)
    yp.columns= ['id','gender','age','diagnosis']
    yp.id = pd.to_numeric(yp.id)
    yp = yp[(yp['diagnosis'] < 3)] 
    ynew = merge(fn2,yp,left_on='id',right_on='id',how='inner')
    filenames = ynew['filename'].values.tolist()
    return(filenames)

In [4]:
filenames = select_filenames(datadir="/media/windows-share/utrecht_eeg",
                            labloc = "/home/vincent/estep/data/utrecht_labels.csv")

In [5]:
def getdata(datadir,labloc,multivar,timecol,printfilenames,filenames,maxtslength=None):
    Nfiles = len(filenames) # number of files
    # Investigate what format the first file has by trying out a variety of reading attempts
    path = datadir + '/' + filenames[0]
    delimiter = [None,','] #possible delimiter values
    skiprows=[0,1]
    ntests = len(delimiter)*len(skiprows)
    df = pd.DataFrame(index=range(ntests),columns=['delimiter','skiprows','nrow','ncol','first cell'])
    cnt = 0
    for di in delimiter:
        for si in skiprows:
            df['delimiter'][cnt] = di
            df['skiprows'][cnt] = si
            try:
                F1 = np.loadtxt(fname=path,delimiter=di,skiprows=si)
                df['nrow'][cnt] = F1.shape[0]
                df['ncol'][cnt] = F1.shape[1]
                df['first cell'][cnt] = F1[0,1]
            except:
                df['nrow'][cnt] = 0
                df['ncol'][cnt] = 0
                df['first cell'][cnt] = 0
            cnt = cnt + 1
    # df is now a dataframe with information to help identify how the data should be loaded
    # load one file based on the extracted information on fileformat
    form = df[df.nrow == max(df.nrow)] # extraction procedure that resulted in the largest number of rows is the best
    if form.shape[0] > 1:
        form = df[df.ncol == max(df.ncol)] # extraction procedure that resulted in the largest number of columns
    if maxtslength != None:
        X = np.zeros((Nfiles,maxtslength,form.ncol)) # hardcoded expected datadimensions
    else:
        if type(labloc) == int:
            X = np.zeros((0,form.ncol,1)) # hardcoded expected datadimensions
            if labloc == 0:
                y = np.zeros((0,1))
            else: 
                y = np.zeros((form.ncol,1))
        else:
            X = np.zeros((0,form.ncol,1))
            y = np.zeros((form.ncol,1))
    filenamelist = list()
    jj = j_rowused = j_fileused = 0 # setting counters
    while jj < Nfiles:
        if printfilenames:
            print(filenames[jj],end=' ')
        path = datadir + '/' + filenames[jj]
        if (form['delimiter'] == ',').bool():
            F2 = np.loadtxt(fname=path,delimiter=',',skiprows=int(form['skiprows']))
        else:
            F2 = np.loadtxt(fname=path,delimiter=None,skiprows=int(form['skiprows']))
        if maxtslength != None: # Modify F2 to meet expected dimantions:
            if F2.shape[0] >= maxtslength:
                F2 = F2[0:maxtslength,]
            elif F2.shape[0] < maxtslength:
                jj += 1
                print(" NOT ENOUGH DATA")
                continue # we are not interested in files with less than maxtslength values
        ## Extract label (y)
        labtype = 'int'
        if jj == 0 and type(labloc) == str: #for Utrecht we only want to get the y once
            y = pd.read_csv(labloc, sep=',',header=0)
        elif type(labloc) == int: # For UCR we want to get all y
            if labloc == 0:
                tmpa = np.array(F2[:,0],dtype=labtype,ndmin=2).transpose()
                y = np.vstack((y,tmpa))
            elif labloc == 1:
                y = np.vstack((y,np.array(F2[0,:], dtype=labtype).transpose()))
        ## Extract data (X)
        if type(labloc) == str:
            if timecol == False:
                X[j_fileused,:,:] = F2.transpose()
            else:
                X[j_fileused,:,:] = F2    
        elif type(labloc) == int:
            tmpp = np.reshape(F2,(F2.shape[0],F2.shape[1],1))
            X = np.vstack((X,tmpp))
            j_rowused += F2.shape[0]-1
        ## Remember filenames
        filenamelist.append(filenames[jj])
        jj += 1
        j_rowused += 1
        j_fileused += 1
    return(X, y, filenamelist, Nfiles)

In [6]:
# get data
if studyname == 'Utrecht':
    X, y, filenamelist, Nfiles = getdata(datadir = "/media/windows-share/utrecht_eeg",
                                     labloc = "/home/vincent/estep/data/utrecht_labels.csv",
                                     multivar = True,
                                     timecol=True,printfilenames=False,filenames=filenames,
                                        maxtslength=4000)
elif studyname == 'UCR':
    datadir = "/home/vincent/estep/data/UCR_TS_Archive_2015/50words"
    filenames = listdir(datadir) 
    X, y, filenamelist, Nfiles = getdata(datadir = datadir,
                                     labloc = 0,
                                     multivar = False,
                                     timecol=False,printfilenames=False,filenames=filenames)


/home/vincent/estep/.venv/lib/python3.4/site-packages/ipykernel/__main__.py:30: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [7]:
print(X.shape, y.shape, len(filenamelist), Nfiles)

(272, 4000, 21) (451, 4) 272 272


In [8]:
# For Utrecht data we now need a conversion from y to a y that matches the X
# merge y and filenamelist id, to get labels per file
def filtery(y,filenamelist):
    fn = pd.DataFrame(filenamelist,columns=['filename'])
    fn = fn.filename.str.split('_',expand=True)
    fn.columns= ['id','epoch']
    fn.id = pd.to_numeric(fn.id)
    fn = fn[fn['epoch'] == 'ep1.TRC.txt']
    ypandas = pd.DataFrame(y)
    ypandas.columns= ['id','gender','age','diagnosis']
    ypandas.id = pd.to_numeric(ypandas.id)
    ynew = merge(fn,ypandas,left_on='id',right_on='id',how='inner')
    ybackup = ynew
    y = ynew.as_matrix()
    y = y[:,4] # we are only interest in the diagnosis in column 4
    return(y)

In [9]:
if studyname == 'Utrecht':
    y = filtery(y,filenamelist)  

In [10]:
print(X.shape, y.shape, len(filenamelist), Nfiles)

(272, 4000, 21) (272,) 272 272


In [11]:
Xtrain = X[0:150,:,:]
ytrain = np.array(y[0:150,],dtype='int')
Xtest = X[151:,:,:]
ytest = np.array(y[151:,],dtype='int')

In [12]:
print(Xtrain.shape, ytrain.shape, Nfiles)
print(Xtest.shape, ytest.shape, Nfiles)

(150, 4000, 21) (150,) 272
(121, 4000, 21) (121,) 272


In [13]:
ytrain.dtype

dtype('int64')

In [14]:
if studyname == 'Utrecht':
    ytrain -= 1
    ytest -= 1

# Design and compile some architectures

In [16]:
# Now, lets try to train some models
from keras.models import Sequential
from keras.layers import Merge, Dense, Dropout, Activation, LSTM
from keras.optimizers import SGD
import numpy as np
from keras.utils.np_utils import to_categorical

Using Theano backend.


In [17]:
ytrain_original = ytrain
ytest_original = ytest
ytrain = to_categorical(ytrain) #np.squeeze(
ytest = to_categorical(ytest) #np.squeeze(

In [18]:
timesteps = Xtrain.shape[1]
if studyname == 'UCR':
    data_dim = 1
    nb_classes = 51
else:
    data_dim = 21
    nb_classes = 2

In [19]:
if studyname == 'UCR':
    model = Sequential()
    # Dense(64) is a fully-connected layer with 64 hidden units.
    idim = timesteps
    model.add(Dense(64, input_dim=idim, init='uniform'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, init='uniform'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, init='uniform'))
    model.add(Activation('softmax'))
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

In [20]:
Xtrain.shape

(150, 4000, 21)

In [21]:
# let's try out another model
model2 = Sequential()
model2.add(LSTM(32, return_sequences=True,
               input_shape=(Xtrain.shape[1], Xtrain.shape[2])))  # returns a sequence of vectors of dimension 32
model2.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model2.add(LSTM(32))  # return a single vector of dimension 32
model2.add(Dense(nb_classes, activation='softmax'))
model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [22]:
model2.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 4000, 32)      6912        lstm_input_1[0][0]               
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 4000, 32)      8320        lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 32)            8320        lstm_2[0][0]                     
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 2)             66          lstm_3[0][0]                     
Total params: 23618
_______________________________________________________________________

# Fit the architectures / models

In [23]:
#Fit model and store history of the fitting process
if studyname == 'UCR':
    Xtrain_tmp  = Xtrain
    Xtrain_tmp = np.reshape(Xtrain,(Xtrain.shape[0],Xtrain.shape[1]))
    History = model.fit(Xtrain_tmp, ytrain,
              nb_epoch=5,batch_size=20,validation_split=0.2)
#else:
#   History = model.fit(X_train, y_train,
#            nb_epoch=5,batch_size=20,validation_split=0.2)


In [24]:
#Fit model and store history of the fitting process
History2 = model2.fit(Xtrain, ytrain,batch_size=20,
                      nb_epoch=5,verbose=True,validation_split=0.2)

Train on 120 samples, validate on 30 samples
Epoch 1/5
120/120 [==============================] - 61s - loss: 0.6856 - acc: 0.5500 - val_loss: 0.6404 - val_acc: 0.6667
Epoch 2/5
120/120 [==============================] - 60s - loss: 0.5569 - acc: 0.8417 - val_loss: 0.6153 - val_acc: 0.6667
Epoch 3/5
120/120 [==============================] - 59s - loss: 0.4782 - acc: 0.8750 - val_loss: 0.5770 - val_acc: 0.7000
Epoch 4/5
120/120 [==============================] - 64s - loss: 0.4014 - acc: 0.8917 - val_loss: 0.5627 - val_acc: 0.7000
Epoch 5/5
120/120 [==============================] - 68s - loss: 0.3296 - acc: 0.9417 - val_loss: 0.5360 - val_acc: 0.8000


# Evaluate models on test set

In [25]:
score2 = model2.evaluate(Xtest, ytest, batch_size=20,verbose=False)
print('Model 2: ' + str(score2))

Model 2: [0.72132241972221811, 0.61157024596348286]


In [26]:
classes2 = model2.predict_classes(Xtest, batch_size=20)
proba2 = model2.predict_proba(Xtest, batch_size=20)

121/121 [==============================] - 9s     


In [29]:
from sklearn import metrics
print(metrics.confusion_matrix(classes2, ytest_original))
print('kappa ' + str(metrics.cohen_kappa_score(classes2, ytest_original)))


[[62 33]
 [14 12]]
kappa 0.0902255639098


In [30]:
metrics.auc(classes2, ytest_original,reorder=True))

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [ ]:
#print(np.hstack((proba2, np.vstack((classes, ytest_original)).transpose())))
#print(np.hstack((proba2, np.vstack((classes2, ytest_original)).transpose())))

# Plot training process

In [ ]:
#set up conditions for plotting
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# plot loss and accuracy for model 2
#fig, ax1 = plt.subplots()
#ax2 = ax1.twinx()
#LN = len(History.history['val_loss'])
#ax1.plot(range(LN),History.history['val_loss'],'g-')
#ax2.plot(range(LN),History.history['val_acc'],'b-')
#ax1.set_xlabel('epoch')
#ax1.set_ylabel('loss',color='g')
#ax2.set_ylabel('accuracy',color='b')
#plt.title('Model 1')

In [ ]:
# plot loss and accuracy for model 2
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
LN = len(History2.history['val_loss'])
ax1.plot(range(LN),History2.history['val_loss'],'g-')
ax2.plot(range(LN),History2.history['val_acc'],'b-')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss',color='g')
ax2.set_ylabel('accuracy',color='b')
plt.title('Model 2')